# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [137]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

import nltk
nltk.download('vader_lexicon')

from newsapi import NewsApiClient

from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

import pprint as pp

%matplotlib inline


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\15189\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [138]:
# Read your api key environment variable
api_key = os.getenv("news_api")

In [139]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [140]:
# Fetch the Bitcoin news articles
btc_headlines = newsapi.get_everything(
    q="bitcoin",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [141]:
# Fetch the Ethereum news articles
eth_headlines = newsapi.get_everything(
    q="ethereum",
    language="en",
    page_size=100,
    sort_by="relevancy"
)

In [142]:
# Create the Bitcoin sentiment scores DataFrame
btc_sentiments = []

for article in btc_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        btc_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
btc_df = pd.DataFrame(btc_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
btc_df = btc_df[cols]

btc_df.head()

,date,text,compound,positive,negative,neutral
0,2020-11-12,A former Microsoft software engineer from Ukra...,-0.6705,0.064,0.199,0.737
1,2020-12-03,Visa has partnered with cryptocurrency startup...,0.6369,0.162,0.000,0.838
2,2020-11-12,PayPal is bringing its newly-announced support...,0.2144,0.053,0.000,0.947
3,2020-11-20,"In November 2017, after an absolutely massive,...",0.2023,0.050,0.000,0.950
4,2020-12-06,"Unlike ‘conventional’ cryptocurrencies, a cent...",0.0000,0.000,0.000,1.000


In [143]:
# Create the ethereum sentiment scores DataFrame
eth_sentiments = []

for article in eth_headlines["articles"]:
    try:
        text = article["content"]
        date = article["publishedAt"][:10]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        pos = sentiment["pos"]
        neu = sentiment["neu"]
        neg = sentiment["neg"]
        
        eth_sentiments.append({
            "text": text,
            "date": date,
            "compound": compound,
            "positive": pos,
            "negative": neg,
            "neutral": neu
            
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
eth_df = pd.DataFrame(eth_sentiments)

# Reorder DataFrame columns
cols = ["date", "text", "compound", "positive", "negative", "neutral"]
eth_df = eth_df[cols]

eth_df.head()

,date,text,compound,positive,negative,neutral
0,2020-11-12,PayPal is bringing its newly-announced support...,0.2144,0.053,0.0,0.947
1,2020-11-23,FILE PHOTO: Representation of the Ethereum vir...,0.0000,0.000,0.0,1.000
2,2020-11-23,FILE PHOTO: Representation of the Ethereum vir...,0.0000,0.000,0.0,1.000
3,2020-11-23,LONDON (Reuters) - Digital currencies Ethereum...,0.4215,0.088,0.0,0.912
4,2020-12-07,NEW YORK (Reuters) - Institutional investors p...,0.1779,0.052,0.0,0.948


In [144]:
# Describe the Bitcoin Sentiment
btc_df.describe()

,compound,positive,negative,neutral
count,98.000000,98.000000,98.000000,98.000000
mean,0.146037,0.054235,0.020490,0.925286
std,0.333504,0.060820,0.053286,0.079791
min,-0.946800,0.000000,0.000000,0.637000
25%,0.000000,0.000000,0.000000,0.858500
50%,0.000000,0.050000,0.000000,0.948000
75%,0.416600,0.117000,0.000000,1.000000
max,0.811800,0.198000,0.363000,1.000000


In [145]:
# Describe the Ethereum Sentiment
eth_df.describe()

,compound,positive,negative,neutral
count,97.000000,97.000000,97.000000,97.000000
mean,0.223294,0.075629,0.021722,0.902660
std,0.360931,0.077265,0.045489,0.087585
min,-0.670500,0.000000,0.000000,0.653000
25%,0.000000,0.000000,0.000000,0.849000
50%,0.214400,0.074000,0.000000,0.912000
75%,0.510600,0.132000,0.000000,1.000000
max,0.883400,0.347000,0.196000,1.000000


### Questions:

Q: Which coin had the highest mean positive score?

A: Ethereum had a higher mean positive score (.077948 vs .019776).

Q: Which coin had the highest compound score?

A: Ethereum had a higher mean compound score (.231637 vs 0.151863) and the highest compound score (.883400 vs .811800).

Q. Which coin had the highest positive score?

A: Ethereum had a highest positive score (.347000 vs .198000).

---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [146]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [147]:
# Expand the default stopwords list if necessary
sw = set(stopwords.words('english'))
sw_addon = {'a', 'to', 'for','than','', 'char', 'chars'}

In [148]:
# Complete the tokenizer function
lemmatizer = WordNetLemmatizer()

def tokenizer(text):
    tokens=[]
    """Tokenizes text."""
    # Create a list of the words
    words_list = word_tokenize(text)

    # Convert the words to lowercase
    first_result = [word.lower() for word in words_list]

    # Remove the punctuation
    regex = re.compile('[^a-zA-Z]')
    re_clean = [regex.sub('',sentences) for sentences in first_result]
    
    # Remove the stop words
    words = [word for word in re_clean if word not in sw.union(sw_addon)]
    
    # Lemmatize Words into root words
    for word in words:
        tokens.append(lemmatizer.lemmatize(word))
    
    return tokens

In [149]:
# Create a new tokens column for bitcoin
for article in btc_df:
    btc_df['tokens'] = [tokenizer(text) for text in btc_df['text']]
btc_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2020-11-12,A former Microsoft software engineer from Ukra...,-0.6705,0.064,0.199,0.737,"[former, microsoft, software, engineer, ukrain..."
1,2020-12-03,Visa has partnered with cryptocurrency startup...,0.6369,0.162,0.000,0.838,"[visa, partnered, cryptocurrency, startup, blo..."
2,2020-11-12,PayPal is bringing its newly-announced support...,0.2144,0.053,0.000,0.947,"[paypal, bringing, newlyannounced, support, cr..."
3,2020-11-20,"In November 2017, after an absolutely massive,...",0.2023,0.050,0.000,0.950,"[november, absolutely, massive, twomonth, rall..."
4,2020-12-06,"Unlike ‘conventional’ cryptocurrencies, a cent...",0.0000,0.000,0.000,1.000,"[unlike, conventional, cryptocurrencies, centr..."


In [150]:
# Create a new tokens column for ethereum
for article in eth_df:
    eth_df['tokens'] = [tokenizer(text) for text in eth_df['text']]
eth_df.head()

,date,text,compound,positive,negative,neutral,tokens
0,2020-11-12,PayPal is bringing its newly-announced support...,0.2144,0.053,0.0,0.947,"[paypal, bringing, newlyannounced, support, cr..."
1,2020-11-23,FILE PHOTO: Representation of the Ethereum vir...,0.0000,0.000,0.0,1.000,"[file, photo, representation, ethereum, virtua..."
2,2020-11-23,FILE PHOTO: Representation of the Ethereum vir...,0.0000,0.000,0.0,1.000,"[file, photo, representation, ethereum, virtua..."
3,2020-11-23,LONDON (Reuters) - Digital currencies Ethereum...,0.4215,0.088,0.0,0.912,"[london, reuters, digital, currency, ethereum,..."
4,2020-12-07,NEW YORK (Reuters) - Institutional investors p...,0.1779,0.052,0.0,0.948,"[new, york, reuters, institutional, investor, ..."


---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [151]:
from collections import Counter
from nltk import ngrams

In [152]:
# Generate the Bitcoin N-grams where N=2
for tokens in btc_df['tokens']:
    for token in tokens:
        big_string = " ".join((big_string,token))

bigrams = ngrams((tokenizer(big_string)), n=2)
bigram_counts = dict(Counter(bigrams))

pp.pprint(bigram_counts)

{('able', 'move'): 1,
 ('abraham', 'lincoln'): 1,
 ('absolutely', 'massive'): 5,
 ('abused', 'steroid'): 5,
 ('academicthink', 'tank'): 1,
 ('acceptance', 'l'): 6,
 ('access', 'ethereum'): 1,
 ('accidental', 'hard'): 1,
 ('according', 'beaconchain'): 1,
 ('according', 'etc'): 1,
 ('according', 'technical'): 1,
 ('account', 'first'): 6,
 ('accrued', 'enough'): 1,
 ('acolyte', 'faceless'): 5,
 ('acquired', 'th'): 1,
 ('acquisition', 'omg'): 1,
 ('across', 'different'): 6,
 ('activate', 'ethereum'): 1,
 ('activate', 'watershed'): 1,
 ('activation', 'ethereum'): 1,
 ('add', 'browse'): 1,
 ('add', 'singapore'): 1,
 ('adding', 'dapp'): 1,
 ('adding', 'eyewatering'): 1,
 ('adding', 'price'): 1,
 ('addition', 'bitcoin'): 1,
 ('address', 'visa'): 5,
 ('adopt', 'bitcoin'): 5,
 ('affordability', 'share'): 1,
 ('ago', 'resulted'): 1,
 ('ahead', 'boosted'): 10,
 ('ahead', 'litecoin'): 1,
 ('ai', 'technology'): 5,
 ('aim', 'expanding'): 1,
 ('aimed', 'increase'): 1,
 ('alive', 'welcome'): 1,
 ('allo

In [153]:
# Generate the Ethereum N-grams where N=2
for tokens in eth_df['tokens']:
    for token in tokens:
        big_string = " ".join((big_string,token))

bigrams = ngrams((tokenizer(big_string)), n=2)
bigram_counts = dict(Counter(bigrams))

pp.pprint(bigram_counts)

{('able', 'move'): 2,
 ('abraham', 'lincoln'): 2,
 ('absolutely', 'massive'): 5,
 ('abused', 'steroid'): 5,
 ('academicthink', 'tank'): 2,
 ('acceptance', 'l'): 7,
 ('access', 'ethereum'): 2,
 ('accidental', 'hard'): 2,
 ('according', 'beaconchain'): 2,
 ('according', 'etc'): 2,
 ('according', 'technical'): 2,
 ('account', 'first'): 7,
 ('accrued', 'enough'): 2,
 ('acolyte', 'faceless'): 5,
 ('acquired', 'th'): 2,
 ('acquisition', 'omg'): 2,
 ('across', 'different'): 7,
 ('activate', 'ethereum'): 2,
 ('activate', 'watershed'): 2,
 ('activation', 'ethereum'): 2,
 ('add', 'browse'): 2,
 ('add', 'singapore'): 2,
 ('adding', 'dapp'): 2,
 ('adding', 'eyewatering'): 2,
 ('adding', 'price'): 2,
 ('addition', 'bitcoin'): 2,
 ('address', 'visa'): 5,
 ('adopt', 'bitcoin'): 5,
 ('affordability', 'share'): 2,
 ('ago', 'resulted'): 2,
 ('ahead', 'boosted'): 10,
 ('ahead', 'litecoin'): 2,
 ('ai', 'technology'): 5,
 ('aim', 'expanding'): 2,
 ('aimed', 'increase'): 2,
 ('alive', 'welcome'): 2,
 ('allo

In [163]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    big_string = ""
    for token_list in tokens:
        for token in token_list:
            big_string = " ".join((big_string,token))
    processed = tokenizer(big_string)
    top_10 = dict(Counter(processed).most_common(10))
    return pd.DataFrame(list(top_10.items()), columns=['word', 'count'])

In [164]:
# Get the top 10 words for Bitcoin
token_count(btc_df.tokens, N=10)

,word,count
0,bitcoin,89
1,reuters,76
2,currency,39
3,photo,33
4,virtual,33
5,taken,31
6,file,29
7,illustration,27
8,reutersdado,27
9,representation,26


In [165]:
# Get the top 10 words for Ethereum
token_count(eth_df.tokens, N=10)

,word,count
0,ethereum,56
1,bitcoin,46
2,reuters,21
3,cryptocurrency,19
4,blockchain,16
5,new,15
6,li,15
7,currency,13
8,week,13
9,asset,13


# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [166]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [ ]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [28]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [29]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [30]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

'Cryptocurrency exchange Binance has resumed trading activity. Users can now cancel open orders, deposit crypto assets into their Binance account, and of course buy and sell cryptocurrencies. You cant withdraw crypto assets to an external wallet just yet, but … [+1191 chars]Bitcoin is now trading at around $8,130, up a whopping 60.84 percent over the past month, with the price surging $3,086.14 over the period.\r\nThe cryptocurrency’s meteoric rise is reminiscent of its rocketing growth in the latter half of 2017, when prices reac… [+4311 chars]Binance has vowed to raise the quality of its security in the aftermath of a hack that saw thieves make off with over $40 million in Bitcoin from the exchange.\r\nThe company — which is widely believed to operate the world’s largest crypto exchange based on tra… [+2269 chars]A new payment network called Flexa is launching today that’ll let you spend cryptocurrencies in physical stores. The technology currently supports bitcoin, ether, bitcoin ca

In [31]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [32]:
# Render the visualization
# YOUR CODE HERE!

In [33]:
# List all Entities
# YOUR CODE HERE!

Binance PERSON
Binance GPE
around $8,130 MONEY
60.84 percent PERCENT
the past month DATE
3,086.14 MONEY
the latter half of 2017 DATE
$40 million MONEY
Bitcoin GPE
Flexa ORG
today DATE
GameStop ORG
Nordstrom NORP
7,000 CARDINAL
Bitcoin GPE
Thursday DATE
Bitcoin GPE
6,000 MONEY
first ORDINAL
November last year DATE
$15 million MONEY
Europol PERSON
three CARDINAL
second ORDINAL
more than €550,000 MONEY
around $615,000 MONEY
last week DATE
’m PERSON
May 7 DATE
about 10,000 CARDINAL
Baltimore GPE
RobbinHood ORG
13 CARDINAL
76,280 MONEY
today DATE
102,310 MONEY
Binance PERSON
last week's DATE
7,000 CARDINAL
roughly $40 million MONEY
Monday DATE
Binance GPE
one CARDINAL
about 7,000 CARDINAL
around $40 million MONEY
Bloomb PERSON
Tether PERSON
New Yorks GPE
month DATE
California GPE
Maine GPE
New York GPE
Albany GPE
Peter da Silva PERSON
the National Academy of Sciencies ORG
1965 DATE
Bitcoin BTC ORG
Tether ORG
Bitcoin GPE
Bitcoin GPE
the Wall Street Market FAC
European NORP
U.S. GPE
Millions 

---

## Ethereum NER

In [34]:
# Concatenate all of the bitcoin text together
# YOUR CODE HERE!

'President Trump tweeted insults at Twitter again this morning, but this time Jack took the conversation off platform to the White House. In other news, a controversial Census question creates some strange bedfellows, and a "blockchain bandit" is pilfering mil… [+1705 chars]After announcing that they were launching a managed blockchain service late last year, Amazon Web Services is now opening that service up for general availability.\r\nIt was only about five months ago that AWS chief executive Andy Jassy announced that the compa… [+1220 chars]Captain Kirk and neo-Dadaists. Repugnant markets and legendary cryptographers. “Digital couture” auctioned by CryptoKitties developers. Distributed autonomous art organizations. A keynote speech looking back from 2047 at the near-apocalypse of 2026, from whic… [+5265 chars]So long as cryptocurrency exists, so too will the extraordinarylengths to which thieves will go to try to steal it. Unfortunately, that also includes preying on weak private k

In [35]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [36]:
# Render the visualization
# YOUR CODE HERE!

In [37]:
# List all Entities
# YOUR CODE HERE!

Trump PERSON
Twitter GPE
this morning TIME
Jack PERSON
the White House ORG
late last year DATE
Amazon Web Services ORG
only about five months ago DATE
Andy Jassy PERSON
Kirk PERSON
neo-Dadaists NORP
Digital ORG
CryptoKitties ORG
2047 DATE
2026 DATE
one CARDINAL
millions CARDINAL
7,000 CARDINAL
Bitcoin GPE
Thursday DATE
Bitcoin GPE
6,000 MONEY
first ORDINAL
November last year DATE
Ether PERSON
ETH ORG
ETH ORG
Vitalik Buterin PERSON
Twitter GPE
Ethereum GPE
early this morning TIME
April 30 DATE
201 CARDINAL
chars]Have PERSON
first ORDINAL
SEC ORG
CoinDesk Korea GPE
over $6.1 million MONEY
Ethereum GPE
Independent Security Evaluators ORG
ISE ORG
ETH ORG
just 7 percent PERCENT
a third CARDINAL
Chainanalysis GPE
chars]Cisco CVE-2019-1804 PERSON
Cisco GPE
9000 PRODUCT
Cisco PERSON
hundreds of millions CARDINAL
Ion ORG
Airbnb PERSON
Facebook PERSON
TRON ORG
TRON ORG
Opera ORG
millions of dollars MONEY
Ethereum GPE
Morgan PERSON
Microsoft ORG
Jane Connolly PERSON
Quorum   PERSON
Ethereum GPE
f